In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv("data/NE_Pacific_Validation_all_vessels.csv")

In [5]:
df.head()

,t,Google Search,nn_max_label,nn_max_sublabel_fishing,Label Dropdown,Expert label,Notes,Person,callsign,nn_max_sub_score,...,callsign.1,num_callsign,gap_hours,num_gaps,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,spoofing_hours,num_overlapping_segs
0,367561870,https://www.google.com/webhp?sourceid=chrome-i...,Tug_Pilot,Trawlers,Tug,Tug,http://www.marinetraffic.com/en/ais/details/sh...,David,WDG6904,1.0,...,WDG6904,4,NaN,NaN,NaN,NaN,NaN,NaN,1220.672500,17.0
1,367692120,https://www.google.com/webhp?sourceid=chrome-i...,Tug_Pilot,Trawlers,other_not_fishing,NaN,"Looks almost like tug, ""local vessel"" https://...",Brady,WDI3806,1.0,...,WDI3806,2,NaN,NaN,NaN,NaN,NaN,NaN,1145.834167,17.0
2,367637970,https://www.google.com/webhp?sourceid=chrome-i...,Tug_Pilot,Trawlers,Tug,NaN,NaN,Brady,WDH6439,1.0,...,WDH6439,4,NaN,NaN,NaN,NaN,NaN,NaN,964.552778,21.0
3,366993020,https://www.google.com/webhp?sourceid=chrome-i...,Tug_Pilot,Trawlers,Tug,NaN,NaN,Brady,WDC2428,1.0,...,WDC2428,1,NaN,NaN,NaN,NaN,NaN,NaN,1180.920556,9.0
4,338164283,https://www.google.com/webhp?sourceid=chrome-i...,Tug_Pilot,Trawlers,no_idea_what_it_is,NaN,no picture,Brady,NaN,1.0,...,NaN,1,350.242778,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_all = pd.read_csv('data/permits.csv', na_filter=False,error_bad_lines=False)

In [31]:
import roman

def normalize_shipname(n):
    
    n = str(n)
    if n == None:
        return ""
    
    # Get rid of roman numerals
    n = n.replace("  "," ")
    n = n.replace("  "," ")
    n = n.replace("  "," ")
    n = n.replace("  "," ") # get rid of extra spaces
    n = n.upper()
    # if the vessel starts with "no. or no53"
    if n[0:3]=="NO.":
        n = n[3:]
        
    if n[0:2]=="NO" and n[2].isdigit():
        n = n[2:]
    
    vs = n.split(" ")
    
    move_to_end = []
    for i in range(len(vs)):
        try:
            try:
                vs[i] = roman.fromRoman(vs[i])
            except:
                pass
            vs[i] = str(int(vs[i]))
            move_to_end.append(i)       
        except:
            pass
    
    if len(move_to_end) == 1:
        vs += [vs.pop(move_to_end[0])]
    
    n = "".join(vs)
    
    n = n.replace(" ","")

    n = n.replace(".",""); #get rid of periods
    n = n.replace("'",""); # get rid of single quotes
    n = n.replace("F/B",''); # get rid of all F/B
    n = n.replace("F/V",''); # get rid of all F/V
    n = n.replace("R/V",''); # get rid of all F/V
    n = n.replace("FV",''); # get rid of all FV
    n = n.replace("#",''); # get ride of #
    n = n.replace("-",''); # get ride of -
    n = n.replace("&",'AND'); # get ride of -
    return n


In [8]:
df_all["norm_shipname"] = df_all['VesselName'].apply(normalize_shipname)
df["norm_shipname"] = df['shipname'].apply(normalize_shipname)


In [13]:
result = pd.merge(df, df_all, on='norm_shipname', how='left')

In [14]:
result.to_csv("ne_pac_mached_noaagroundfish.csv")

In [16]:
headers = '''Vessel ID	AN	          10	     1   -      10	
Vessel Name	AN	50	11	-	60
Call Sign	AN	8	61	-	68
Official Number	AN	10	69	-	78
IMO Number	AN	30	79	-	108
Hull Number	AN	30	109	-	138
Hull Identification Number	AN	30	139	-	168
Vessel Service	CH	30	169	-	198
Flag	CH	2	199	-	200
Self Propelled Indicator	Logical	1	201	-	201 
Registered Gross Tons	DS	7	202	-	208
Registered Net Tons	DS	7	209	-	215
Registered Length	DS	7	216	-	222
Registered Breadth	DS	6	223	-	228
Registered Depth	DS	6	229	-	234
ITC Gross Tons	DS	7	235	-	241
ITC Net Tons	DS	7	242	-	248
ITC Length	DS	7	249	-	255
ITC Breadth	DS	6	256	-	261
ITC Depth	DS	6	262	-	267
***Dead Weight Tons	DS	7	268	-	274
***Dead Weight Tons Measure Unit	CH	40	275   -	314
Measuring Organization Name	CH	50	315	-	364
Hailing Port	CH	50	365	-	414
Hailing Port State	CH	2	415	-	416	
Hailing Port Country	CH	50	417	-	466
Coastwise Unrestricted	CM	1	467	-	467
Limited Coastwise Bowaters Only	CM	1	468	-	468
Limited Coastwise Restricted MARAD Waiver	CM	1	469	-	469
 Limited Coastwise Oil Spill Response Only	CM	1	470 	-	470
Limited Coastwise Under Charter to Citizen	CM	1	471	-	471
Limited Coastwise Trade Fish Products Only	CM	1	472	-	472
Fishery	CM	1	473	-	473
Fishery Only	CM	1	474	-	474
Recreation	CM	1	475	-	475
Limited Recreation Great Lakes Use Only	CM	1	476	-	476
Registry	CM	1	477	-	477
Limited Registry Cross Border Financing	CM	1	478	-	478
Limited Registry No Foreign Voyage by Sea	CM	1	479	-	479
Limited Registry Trade with Canada Only	CM	1	480	-	   480
Great Lakes	CM	1	481	-	481
Builder	CH	80	 482	-	 561
Shipyard 	CH	50           562 	-    611
Build Year	CH	4	612	-	615
Vessel Complete Build City	CH	50	616	-	665
Vessel Complete Build State	CH	2	666	-	667
Vessel Complete Build Province	CH	50	668	-	717
Vessel Complete Build Country	CH	64	718	-	781
Vessel Hull Build City	CH	50	782	-	831
Vessel Hull Build State	CH	2	832	-	833
Vessel Hull Build Province	CH	50	834	-	883
Vessel Hull Build Country	CH	64	884	-	947
Party ID	AN	10	948	- 	957
Organization Name	CH	80	958	-	1037
Organization Type	CH	30	1038  -	1067
Person First Name	CH	30	1068   -	1097
Person Middle Name	CH	50	1098   -	1147
Person Last Name	CH	50	1148   -	1197
Person Name Suffix	CH 	10	1198	-	1207
Address Line 1	CH	255	1208	-	1462
Address Line 2	CH	255	1463	-	1717
Address Line 3	CH	255	1718	-	1972
Address Line 4	CH	255	1973	-	2227
City	CH	30	2228	-	2257
State	CH	2	2258	- 	2259
Province	CH	30	2260	-	2289
Country	CH	64	2290	-	2353
Postal Code	CH	30	2354	-	2383
Main HP Ahead	 IS	7	2384	-	2390
Main HP Astern	 IS	7	2391 	-	2397
Propulsion Type	CH	30	2398	 -	2427
Hull Material	CH	30	2428	-	2457
Hull Configuration	CH	30	2458	-	2487
Hull Shape	CH	30	2488	-	2517
COD Status	CH	30	2518	-	2547
COD issue date	CH	10	2548	-	2557
COD expire date	CH	10	2558	-	2567
Filler	CH	1	2568	-	2568'''.split("\n")

for i in range(len(headers)):
    headers[i] = headers[i].split("\t")[0]

In [17]:
df_all = pd.read_csv('data/VesDocDec16tab.txt',sep='\t',header=0, names=headers, na_filter=False,error_bad_lines=False)

Skipping line 45738: expected 78 fields, saw 79

Skipping line 144888: expected 78 fields, saw 79

Skipping line 190597: expected 78 fields, saw 79

Skipping line 208464: expected 78 fields, saw 79

Skipping line 252491: expected 78 fields, saw 79

Skipping line 342041: expected 78 fields, saw 79

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
df_fishing = df_all.loc[df_all['Vessel Service'].isin(['Commercial Fishing Vessel','Fish Processing Vessel'])]


In [25]:
df_fishing_valid = df_fishing.loc[df_fishing['COD Status'] == "Valid"].rename(columns={'Call Sign': 'callsign'})

In [26]:
df_fishing_valid["norm_shipname"] = df_fishing_valid['Vessel Name'].apply(normalize_shipname)


In [27]:
result2 = pd.merge(result, df_fishing_valid.loc[df_fishing_valid['callsign'] != ""], 
                   on=['callsign','norm_shipname'], how='left')

In [30]:
result2.to_csv("NEPac-noaagroundfish-merchantUS.csv")